In [1]:
import alerts.analysis

In [2]:
%time X, Y, X_prime = alerts.analysis.get_files()

{'queryId': 'f95a6734-31ca-4b9f-ab2a-cb6eb52eb3f3', 'ResponseMetadata': {'RequestId': 'b68949e0-7676-4610-a8ba-94c6a391f668', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b68949e0-7676-4610-a8ba-94c6a391f668', 'content-type': 'application/x-amz-json-1.1', 'content-length': '50', 'date': 'Sat, 25 Jan 2020 23:26:41 GMT'}, 'RetryAttempts': 0}}
2020-01-01 03:13:32.404000+00:00
146
03e816a52bd4dd9843e38f8a7844ea4c.html
fetch_s3
No clicks found in 03e816a52bd4dd9843e38f8a7844ea4c.html
088caeb33b0536d20d6fa2008ce76322.html
fetch_s3
No clicks found in 088caeb33b0536d20d6fa2008ce76322.html
0ff3f316a3d7d7564b68e6b1fe7f7330.html
fetch_s3
1395e620f53b2497792275eca944fd0b.html
fetch_s3
23ac2308cb76ea8c6c71fd959aadd652.html
fetch_s3
No clicks found in 23ac2308cb76ea8c6c71fd959aadd652.html
29032d37db17c432460b1a62228cd166.html
fetch_s3
No clicks found in 29032d37db17c432460b1a62228cd166.html
3cca55942b88c2dd0b786fdd7ffcbadb.html
fetch_s3
No clicks found in 3cca55942b88c2dd0b786fdd7ffcb

In [3]:
import xgboost

In [4]:
from bert_embedding import BertEmbedding
bert_embedding = BertEmbedding()

In [5]:
%time result = bert_embedding(X)

CPU times: user 13min 23s, sys: 49.3 s, total: 14min 12s
Wall time: 14min 11s


In [9]:
from collections import Counter

In [11]:
tf = Counter()
for i in result:
    tf.update(i[0])

In [18]:
N = sum(tf.values())

In [47]:
from sklearn.feature_extraction import FeatureHasher

In [57]:
h = FeatureHasher(n_features=100, input_type="string")

In [43]:
s = result[0]

In [72]:
import numpy

def s_from_w(s):
    words = s[0]
    embedding = numpy.array(s[1])
    embedding = numpy.concatenate((embedding, h.transform(words).toarray()), axis=1)
    weight = numpy.array([1/(1+tf[x]/N)/len(words) for x in words])
    return weight.dot(embedding)

In [117]:
SX = numpy.array([s_from_w(x) for x in result])

In [118]:
SX.shape

(5346, 868)

In [119]:
from sklearn.decomposition import TruncatedSVD

In [120]:
svd = TruncatedSVD(n_components=1, n_iter=7, random_state=42)

In [122]:
%time svd.fit(SX)

CPU times: user 438 ms, sys: 8.02 ms, total: 446 ms
Wall time: 228 ms


TruncatedSVD(algorithm='randomized', n_components=1, n_iter=7,
       random_state=42, tol=0.0)

In [123]:
u = svd.components_

In [88]:
#v = numpy.array([ v - (v.dot(u.transpose()))*u        for v in SX])

In [103]:
#v.shape

In [124]:
v2 = SX - SX.dot(u.transpose())*u

In [104]:
#v[1,0,3]

Algorithm 1 Sentence Embedding
Input: Word embeddings {vw : w ∈ V}, a set of sentences S, parameter a and estimated probabilities {p(w) : w ∈ V} of the words.
Output: Sentence embeddings {vs : s ∈ S}
1: for all sentence s in S do
2: v_s ← 1 / |s| Sum_w∈s a / (a+p(w)) v_w
3: end for
4: Form a matrix X whose columns are {vs : s ∈ S}, and let u be its first singular vector
5: for all sentence s in S do
6: vs ← vs − uu>vs
7: end for

In [1]:
import xgboost as xgb

In [127]:
#copypasta from manual
### load data in do training
dtrain = xgb.DMatrix(v2, Y)

In [3]:
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 20

In [131]:
dtrain.save_binary("train.dmatrix")

In [2]:
dtrain = xgb.DMatrix("train.dmatrix")

[00:33:02] 5346x868 matrix with 4640328 entries loaded from train.dmatrix


In [11]:
print('running cross validation')
# do cross validation, this will print result out as
# [iteration]  metric_name:mean_value+std_value
# std_value is standard deviation of the metric
result = xgb.cv(param, dtrain, num_round, nfold=5,
       metrics={'auc'}, seed=0,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])

running cross validation
[0]	train-auc:0.533422+0.00504675	test-auc:0.501996+0.00595748
[1]	train-auc:0.626185+0.0479941	test-auc:0.526923+0.0271428
[2]	train-auc:0.733682+0.0304535	test-auc:0.564882+0.0333571
[3]	train-auc:0.790488+0.0216768	test-auc:0.571301+0.0457722
[4]	train-auc:0.819986+0.0157153	test-auc:0.592443+0.0230354
[5]	train-auc:0.848636+0.0101834	test-auc:0.62169+0.0280735
[6]	train-auc:0.871389+0.0145809	test-auc:0.624308+0.0291835
[7]	train-auc:0.895109+0.0121665	test-auc:0.644368+0.0538952
[8]	train-auc:0.912586+0.0126259	test-auc:0.636505+0.0449627
[9]	train-auc:0.922347+0.00871506	test-auc:0.636228+0.0466585
[10]	train-auc:0.936209+0.0113212	test-auc:0.63346+0.0380729
[11]	train-auc:0.947258+0.0132368	test-auc:0.628248+0.0374894
[12]	train-auc:0.956417+0.0102649	test-auc:0.611765+0.036503
[13]	train-auc:0.965197+0.00848947	test-auc:0.603831+0.0406729
[14]	train-auc:0.972116+0.00709517	test-auc:0.596683+0.0374702
[15]	train-auc:0.978128+0.00750296	test-auc:0.586471+

0.6443676

In [19]:
import hyperopt

from hyperopt import hp, tpe, Trials
from hyperopt.fmin import fmin

In [20]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
# https://www.kaggle.com/eikedehling/tune-and-compare-xgb-lightgbm-rf-with-hyperopt

In [26]:
trials = Trials()


In [ ]:
def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    params.update(param)
    
    result = xgb.cv(params, dtrain, num_round, nfold=9,
       metrics={'auc'}, seed=0)
    
    score = result['test-auc-mean'].max()

    return -score

space = {
    'max_depth': hp.quniform('max_depth', 2, 12, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            trials=trials,
            max_evals=200)

 84%|████████▍ | 168/200 [16:24<15:15, 28.59s/trial, best loss: -0.6552801111111111]

In [ ]:
best

In [23]:
best

{'colsample_bytree': 0.4998771532981213,
 'gamma': 0.0635933855060849,
 'max_depth': 5.0}